In [ ]:
# Check GPU
!nvidia-smi

# Install libs (headless OpenCV avoids GUI issues in Colab)
!pip -q install "ultralytics>=8.3.0" opencv-python-headless onnxruntime


In [ ]:
# --- Fixed Cell 2: robust demo video + quick inference ---
from ultralytics import YOLO
import urllib.request, os, pathlib

model = YOLO("yolov8n.pt")   # already cached

# Try a few known-good sample videos (small)
urls = [
    # OpenCV sample traffic clip
    "https://github.com/opencv/opencv/raw/master/samples/data/vtest.avi",
    # Another OpenCV sample
    "https://github.com/opencv/opencv_extra/raw/master/testdata/highgui/video/big_buck_bunny.avi",
]

dest = "demo.mp4"  # we'll rename whatever we get to .mp4 for convenience
ok = False
for i, u in enumerate(urls):
    tmp = f"demo_src_{i}" + pathlib.Path(u).suffix
    try:
        print(f"Downloading: {u}")
        urllib.request.urlretrieve(u, tmp)
        # normalize extension -> .mp4 (Ultralytics handles multiple formats, but keep one name)
        os.rename(tmp, dest)
        ok = True
        break
    except Exception as e:
        print(f"Failed: {e}")

if not ok:
    raise RuntimeError("Could not fetch a sample video. As a fallback, upload a short dashcam/traffic clip and set dest='yourfile.mp4'.")

# Inference on video, save annotated frames to runs/detect/predict*/
results = model.predict(source=dest, conf=0.25, save=True)

# Show where results were saved
print("Saved to:", results[0].save_dir if results else "No results")


In [ ]:
from ultralytics import YOLO
import torch

device = 0 if torch.cuda.is_available() else 'cpu'
model = YOLO("yolov8n.pt")

# Fast sanity fine-tune: 5–10 epochs on small sample
model.train(data="coco128.yaml", epochs=5, imgsz=640, batch=16, device=device)

# Path to best weights
best = model.ckpt_path if hasattr(model, "ckpt_path") else "runs/detect/train/weights/best.pt"
print("Best weights:", best)

# Quick validation
model.val()

In [ ]:
# --- Cell: find newest best.pt, export to ONNX, show paths ---
import glob, os, time, pathlib
from ultralytics import YOLO

# Find the latest best.pt under runs/detect/*/weights/best.pt
cands = glob.glob("runs/detect/*/weights/best.pt")
assert cands, "No best.pt found. Did training finish? Check runs/detect/*/weights/"
best_pt = max(cands, key=os.path.getmtime)
print("Using best weights:", best_pt)

# Export to ONNX next to the .pt
m = YOLO(best_pt)
onnx_path = m.export(format="onnx", opset=12, dynamic=True)
print("Exported ONNX:", onnx_path)

# If you want TorchScript too (optional):
# ts_path = m.export(format="torchscript")
# print("Exported TorchScript:", ts_path)


In [ ]:
from google.colab import files
files.download("runs/detect/train/weights/best.pt")
files.download("runs/detect/train/weights/best.onnx")
